In [56]:
# import some packages
import pandas as pd
from tflearn.data_utils import pad_sequences
from collections import Counter
import random
import numpy as np
import h5py
import pickle
import codecs
print("import package successful...")

import package successful...


#### 读取训练数据

In [57]:
# 读取训练数据
base_path='/data/chenhy/data/ieee_zhihu_cup/'
train_data_x=pd.read_csv(base_path+'question_train_set3.txt',sep='\t', encoding="utf-8")
train_data_y=pd.read_csv(base_path+'question_topic_train_set3.txt',sep='\t', encoding="utf-8")

train_data_x=train_data_x.fillna('')
train_data_y=train_data_y.fillna('')

print("train_data_x:",train_data_x.shape)
print("train_data_y:",train_data_y.shape)

train_data_x: (2999967, 5)
train_data_y: (2999967, 2)


In [58]:
# 组成新的文件

#连接title和desc
train_data_x['title_desc_char'] = train_data_x.apply(lambda t: t['title_char'] + ',SEP,' + t['desc_char'], axis=1)
train_desc = pd.concat([train_data_x['title_desc_char'], train_data_y['topic_ids']],axis=1)
del train_data_x
del train_data_y

In [59]:
#创建word2index表和embedding表，此处word为char
word_embedding_object=open(base_path+'unused_current/char_embedding.txt')
lines_wv=word_embedding_object.readlines()
word_embedding_object.close()

#初始化为长度为3的列表，分别对应【填充、未知、连接】
char_list=[]
char_list.extend(['PAD','END','SEP'])
PAD_ID=0 
UNK_ID=1
EMBED_SIZE = 100
bound = 0.5
embedding = [None] * (int(lines_wv[0].split(' ')[0]) + 3) #长度
embedding[0] = np.zeros(100)
embedding[1] = np.random.rand(100)
embedding[2] = embedding[0] + 0.1

#对 embedding 文件中，建立 embedding 列表
for i, line in enumerate(lines_wv):
    if i==0: continue
    char_embedding_list=line.split(" ")
    char_token=char_embedding_list[0]
    char_list.append(char_token)
    embedding[i+2] = char_embedding_list[1:1+EMBED_SIZE]

embedding = np.array(embedding,dtype=np.float32)

vocab_path=base_path+'vocab.txt'
vocab_char_object=open(vocab_path,'w')

#建立word2index表
word2index={}
for i, char in enumerate(char_list):
    if i<10:print(i,char)
    word2index[char]=i
    vocab_char_object.write(char+"\n")
vocab_char_object.close()
print("vocabulary of char generated....")

0 PAD
1 END
2 SEP
3 </s>
4 c17
5 c101
6 c11
7 c4
8 c147
9 c85
vocabulary of char generated....


In [60]:
 # generate labels list, and save to file system 
c_labels=Counter()
train_data_y_small=train_desc['topic_ids'][0:100000]#.sample(frac=0.1)
for index, topic_ids in enumerate(train_data_y_small):
    topic_list=topic_ids.split(',')
    c_labels.update(topic_list)

label_list=c_labels.most_common()
label2index={'PAD': 0, 'END': 1, 'GO': 2}

label_target_object=open(base_path+'label_set.txt','w')
for i, label_freq in enumerate(label_list):
    label,freq=label_freq
    label2index[label]=i+3
    label_target_object.write(label+"\n")
    if i<20: print(label,freq)
label_target_object.close()
print("generate label dict successful...")

7476760589625268543 2308
4697014490911193675 1746
-4653836020042332281 1579
-8175048003539471998 1475
-8377411942628634656 1382
-7046289575185911002 1338
-5932391056759866388 1283
2787171473654490487 1145
-7129272008741138808 1085
2587540952280802350 1079
-4931965624608608932 1079
-6748914495015758455 1049
-5513826101327857645 993
2347973810368732059 970
9069451131871918127 958
-8132909213241034354 904
-3517637179126242000 867
-5872443091340192918 834
-3522198575349379632 830
1127459907694805235 829
generate label dict successful...


#### 生成新数据

In [61]:
Seq2Seq_label_length = 5

def new_row(row):
    x, y = row['title_desc_char'].strip(), row['topic_ids'].strip()
    # x: c324,c39,c40,c155,c180,c180,c181,c17,c4,c1153
    # y: 7739004195693774975,3738968195649774859
    x = [word2index.get(c, 0) for c in x.split(',')]
    if len(x) > 100: x = x[:100]
    else: 
        for i in range(len(x), 100): x.append(0)
    y = y.split(',')
    y_multihot_list = [label2index['PAD']] * Seq2Seq_label_length
    y_decoder_input = [label2index['PAD']] * Seq2Seq_label_length
    y_decoder_input[0] = label2index['GO']
    for i, label in enumerate(y):
        if i < Seq2Seq_label_length - 1:
            y_multihot_list[i] = label2index[label]
            y_decoder_input[i + 1] = label2index[label] #decoder第一位为空
            
    if len(y) > Seq2Seq_label_length - 1:
        y_multihot_list[Seq2Seq_label_length - 1] = label2index['END']
    else:
        y_multihot_list[len(y)] = label2index['END']
    
    if len(y) + 1 > Seq2Seq_label_length - 1:
        y_decoder_input[Seq2Seq_label_length - 1] = label2index['END']
    else:
        y_decoder_input[len(y) + 1] = label2index['END']
    return x, y_multihot_list, y_decoder_input

X_y_multihot_y_decoder = train_desc.apply(new_row, axis=1) 
#一个series，每个元素是3个列表的元组，三个列表分别为x，y_multihot_list, y_decoder_input
X_y_multihot_y_decoder = list(zip(*X_y_multihot_y_decoder))
#3个列表的元组，3个列表分别为x的列表，y_multihot_list的列表, y_decoder_input的列表

#### 保存数据

In [64]:
#保存数据
cache_file_h5py = base_path + 'seq2seq_data.h5'
cache_file_pickle = base_path + 'seq2seq_w2i_l2i.pkl'
f = h5py.File(cache_file_h5py, 'w')
f['train_X'] = X_y_multihot_y_decoder[0][:-20000]
f['test_X'] = X_y_multihot_y_decoder[0][-20000:]
f['train_Y'] = X_y_multihot_y_decoder[1][:-20000]
f['test_Y'] = X_y_multihot_y_decoder[1][-20000:]
f['train_decoder_Y'] = X_y_multihot_y_decoder[2][:-20000]
f['test_decoder_Y'] = X_y_multihot_y_decoder[2][-20000:]
f['embedding'] = embedding
f.close()
# save word2index, label2index
with open(cache_file_pickle, 'ab') as target_file:
    pickle.dump((word2index,label2index), target_file)

In [65]:
f

<Closed HDF5 file>